<a href="https://colab.research.google.com/github/UN-GCPDS/python-gcpds.luker_multiple_annotators/blob/main/notebooks/chocolate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

## Load repo

In [1]:
! git clone https://github.com/UN-GCPDS/python-gcpds.luker_multiple_annotators.git

Cloning into 'python-gcpds.luker_multiple_annotators'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 85 (delta 20), reused 64 (delta 8), pack-reused 0
Receiving objects: 100% (85/85), 17.80 MiB | 13.72 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [2]:
%cd "/content/python-gcpds.luker_multiple_annotators"

/content/python-gcpds.luker_multiple_annotators


## Load libraries

In [19]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import sys
sys.path.append('/content/python-gcpds.luker_multiple_annotators/src')
! pip install umap-learn
from multiple_annotators_models import CKA, LCKA
from utils import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 2.0 MB/s eta 0:00:00


## Load data

In [5]:
df_sens = pd.read_csv('data/datos_sensoriales.csv', index_col=0)
df_fq = pd.read_csv('data/datos_fisi_quim.csv')
df_comb = df_sens.merge(df_fq, left_on='codigo sampler', right_on='CodMuestra')
df_comb

<ipython-input-5-136c44a6aeb5>:1: DtypeWarning: Columns (4,77,78,79,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sens = pd.read_csv('data/datos_sensoriales.csv', index_col=0)


,codigo sampler,hora inicio,hora finalizacion,muestra,codigo evaluador,familia producto,producto,acidez,adherencia,almendra,...,viscosidad aparente ha1 70°c,viscosidad aparente ha7 40°c,viscosidad plastica,viscosidad plastica anton paar,viscosidad plastica brookfield,viscosidad plastica ensayo,viscosidad plastica [duplicado] anton paar,viscosidad plastica [preliminar],viscosidad plastica nca/cma,zaranda <5mm
0,D03221578,2022-04-22 11:06:18,2022-04-22 11:10:32,967.0,160,Dark Chocolate Rutinarios,marcha rich,1.0,NaN,0.0,...,NaN,NaN,NaN,1.8786,NaN,NaN,NaN,NaN,NaN,NaN
1,D03221578,2022-04-22 11:07:58,2022-04-22 11:11:00,967.0,154,Dark Chocolate Rutinarios,match rich,0.0,NaN,0.0,...,NaN,NaN,NaN,1.8786,NaN,NaN,NaN,NaN,NaN,NaN
2,D03221578,2022-04-22 11:10:39,2022-04-22 11:14:52,967.0,967,Dark Chocolate Rutinarios,MATCH RICH,1.0,NaN,0.0,...,NaN,NaN,NaN,1.8786,NaN,NaN,NaN,NaN,NaN,NaN
3,D04221765,2022-04-22 11:10:35,2022-04-22 11:12:07,758.0,160,Dark Chocolate Rutinarios,dark chocolate 70 % organic petitorio,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,D04221765,2022-04-22 11:11:01,2022-04-22 11:12:13,758.0,154,Dark Chocolate Rutinarios,dark chocolate 70% organic petit,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13843,Z09230090,2023-09-08 15:14:48,2023-09-08 15:17:05,400,141,Dark Chocolate Rutinarios,Fark mango,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13844,Z09230090,2023-09-08 19:42:12,2023-09-08 19:45:45,400,101,Dark Chocolate Rutinarios,Dark mango,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13845,Z09230091,2023-09-08 13:54:15,2023-09-08 13:57:35,111,169,Dark Chocolate Rutinarios,Piña,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13846,Z09230091,2023-09-08 15:11:49,2023-09-08 15:16:27,111,101,Dark Chocolate Rutinarios,111,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
fq_vars = ['humedad - determinador halogeno', 'grasa - nmr', 'granulometría - micrometro',
           'viscosidad plastica  anton paar', 'limite de fluidez  anton paar']
sens_vars = ['acidez', 'amargor', 'aroma', 'astringencia', 'dulce', 'dureza',
             'impresion global', 'velocidad fusion']
lista_anotadores = [132, 135, 154, 155, 160, 179]

In [14]:
df_vars_choc, df_ma_choc = create_dfs_impute(df_fq, df_sens, lista_anotadores, fq_vars, sens_vars, 'Chocolate')

In [15]:
df_ma_choc

acidez                          amargor                 ...  \
codigo evaluador    132  135  154  155  160  179     132  135  154  155  ...   
D01211557           1.5  2.5  0.5  1.5  0.0  1.5     5.0  4.0  4.5  5.5  ...   
D01220257           1.5  1.0  0.0  1.0  2.0  1.5     6.5  6.0  2.0  6.0  ...   
D01220258           0.5  2.0  0.0  1.0  1.0  1.5     6.0  5.0  2.0  6.0  ...   
D01220924           3.0  3.0  0.0  1.0  0.0  2.0     6.0  6.5  5.0  7.0  ...   
D01220946           0.0  0.5  0.0  0.0  0.0  0.5     1.0  0.5  0.5  1.0  ...   
...                 ...  ...  ...  ...  ...  ...     ...  ...  ...  ...  ...   
Z12231788           1.5  0.0  1.0  0.0  1.0  1.0     5.0  0.0  3.5  0.0  ...   
Z12232285           2.0  0.0  0.0  1.0  2.0  3.0     4.0  3.0  2.0  6.0  ...   
Z12232287           2.0  0.0  0.0  1.0  2.0  2.0     4.0  3.0  2.0  6.0  ...   
Z12232288           2.0  0.0  0.0  1.0  4.0  3.5     4.0  3.0  2.0  6.0  ...   
Z12232289           2.0  0.0  0.0  1.0  3.0  3.5     4.0  3.0  2.0  6.0  ...   

                 impresion global                 velocidad fusion            \
codigo evaluador              154  155   160  179              132  135  154   
D01211557                     7.0  5.5  4.00  5.5              6.0  5.5  5.0   
D01220257                     7.0  7.0  8.00  6.5              5.0  6.5  4.0   
D01220258                     7.0  7.0  7.75  6.5              5.5  6.5  4.0   
D01220924                     6.5  7.0  8.00  7.0              6.0  5.5  5.0   
D01220946                     6.0  6.0  8.00  6.0              0.0  0.0  0.0   
...                           ...  ...   ...  ...              ...  ...  ...   
Z12231788                     6.5  7.0  6.50  5.0              5.5  5.5  4.0   
Z12232285                     7.0  7.0  8.00  6.0              5.0  6.0  4.0   
Z12232287                     7.0  7.0  7.00  6.0              5.0  6.0  4.0   
Z12232288                     7.0  7.0  8.00  7.0              5.0  6.0  4.0   
Z12232289                     7.0  7.0  8.00  7.0              5.0  6.0  4.0   

                                 
codigo evaluador  155  160  179  
D01211557         5.0  6.0  4.0  
D01220257         5.5  6.0  5.0  
D01220258         5.5  6.0  5.0  
D01220924         6.0  6.0  4.0  
D01220946         4.5  3.0  5.0  
...               ...  ...  ...  
Z12231788         4.0  5.5  4.0  
Z12232285         5.5  0.0  4.0  
Z12232287         5.5  0.0  4.0  
Z12232288         5.5  0.0  4.0  
Z12232289         5.5  0.0  4.0  

[1623 rows x 48 columns]

In [16]:
df_vars_choc

,humedad - determinador halogeno,grasa - nmr,granulometría - micrometro,viscosidad plastica anton paar,limite de fluidez anton paar
D01211557,0.430000,41.476806,25.455227,1.387576,7.747798
D01220257,0.244416,38.751247,25.688024,1.876016,10.992283
D01220258,0.244416,38.751247,25.688024,1.876016,10.992283
D01220924,0.140000,37.918000,25.729318,2.028609,12.100579
D01220946,0.200000,35.125100,27.572950,1.445700,0.518900
...,...,...,...,...,...
Z12231788,0.190000,33.510000,27.000000,5.640800,22.428000
Z12232285,0.244416,38.751247,25.688024,1.876016,10.992283
Z12232287,0.244416,38.751247,25.688024,1.876016,10.992283
Z12232288,0.244416,38.751247,25.688024,1.876016,10.992283


In [23]:
df_ma_choc.loc[:, (slice(None), 132)]

,acidez,amargor,aroma,astringencia,dulce,dureza,impresion global,velocidad fusion
codigo evaluador,132,132,132,132,132,132,132,132
D01211557,1.5,5.0,6.5,3.0,4.5,6.0,5.5,6.0
D01220257,1.5,6.5,5.5,3.5,3.5,5.0,7.0,5.0
D01220258,0.5,6.0,6.0,2.5,4.5,4.0,5.5,5.5
D01220924,3.0,6.0,8.0,3.0,5.0,6.0,7.0,6.0
D01220946,0.0,1.0,6.0,1.0,7.0,0.0,6.0,0.0
...,...,...,...,...,...,...,...,...
Z12231788,1.5,5.0,5.5,3.0,5.5,4.5,5.5,5.5
Z12232285,2.0,4.0,6.0,2.5,5.0,4.0,6.5,5.0
Z12232287,2.0,4.0,6.0,2.5,5.0,4.0,6.5,5.0


# Models

In [25]:
df_ma_choc.max()

                  codigo evaluador
acidez            132                  6.0
                  135                  7.0
                  154                  3.0
                  155                  7.0
                  160                  9.0
                  179                  5.0
amargor           132                  8.0
                  135                  8.5
                  154                  8.0
                  155                  9.0
                  160                  7.0
                  179                  7.0
aroma             132                  8.0
                  135                  7.0
                  154                  8.0
                  155                  7.0
                  160                  7.0
                  179                  7.0
astringencia      132                  6.0
                  135                 10.0
                  154                  6.0
                  155                  5.0
                  1

In [30]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, DotProduct
from sklearn.metrics import mean_absolute_error

In [31]:
gp_vars_tot = {}
for ann in lista_anotadores:
    df_ma_ann = df_ma_choc.loc[:, (slice(None))].dropna()
    y = df_ma_ann.values/10
    scaler_X = MinMaxScaler()
    X = scaler_X.fit_transform(df_vars_choc.values)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    gp_vars = {}
    cons_params = {'constant_value': 1.0,
                   'constant_value_bounds':(1e-10, 100.0)}
    kernels = [
        ConstantKernel(**cons_params) * RBF([1,1,1,1,1], [1e-20,100]) + \
        ConstantKernel(**cons_params) * DotProduct()
    ]
    for i, var in enumerate(sens_vars):
        y_train_var, y_test_var = y_train[:,i], y_test[:,i]
        regressor = GaussianProcessRegressor()

        # Define the param_grid dictionary with different kernel options
        param_grid = {'kernel': kernels}

        grid_search = GridSearchCV(regressor, param_grid, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_absolute_percentage_error'],
                                n_jobs=-1, refit='neg_mean_absolute_error')
        # Perform hyperparameter tuning
        grid_search.fit(X_train, y_train_var)

        # Get the best model
        best_reg = grid_search.best_estimator_

        # Make predictions on the test set
        y_pred = best_reg.predict(X_test)
        mae = mean_absolute_error(y_test_var, y_pred)
        gp_vars[var] = grid_search
    gp_vars_tot[ann] = gp_vars



/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified lower bound 1e-20. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 1 of parameter k1__k2__length_scale is close to the specified lower bound 1e-20. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/

In [34]:
gp_vars_tot[132]['acidez']

GridSearchCV(cv=5, estimator=GaussianProcessRegressor(), n_jobs=-1,
             param_grid={'kernel': [1**2 * RBF(length_scale=[1, 1, 1, 1, 1]) + 1**2 * DotProduct(sigma_0=1)]},
             refit='neg_mean_absolute_error',
             scoring=['neg_mean_absolute_error', 'neg_mean_squared_error',
                      'neg_mean_absolute_percentage_error'])

In [35]:
import pickle

In [37]:
for ann in lista_anotadores:
    for var in sens_vars:
        with open(f'/content/python-gcpds.luker_multiple_annotators/models/chocolate_gps/{ann}_{var}.pkl', 'wb') as f:
            pickle.dump(gp_vars_tot[ann][var], f)